[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/EricBaidoo/GhanaSegNet/blob/main/Colab_Evaluate_and_Package.ipynb)


# Colab: Evaluate Checkpoints & Package Results

This notebook mounts Google Drive, copies checkpoints/results into the Colab runtime, installs dependencies, runs either a quick "Path A" analysis (process existing JSONs) or a full "Path B" evaluation (run `analysis/evaluate_checkpoints_per_class.py` on checkpoints), generates plots and per-class JSON outputs, and packages the small outputs back to Drive.

How to use:
1. Open this notebook in Colab (File > Upload notebook or via `colab.research.google.com` and select this GitHub/Drive file).
2. Run cells sequentially. Use Path A if you already have `results/*.json` in Drive. Use Path B to evaluate checkpoints in Drive (requires GPU and the dataset or access to dataset files).

This notebook will write a zip file to `/content/drive/My Drive/GhanaSegNet_Results/results_per_class_summary_jsons.zip` containing the small JSON and PNG outputs.

## 1) Check for an existing Colab notebook

This section searches the repository for other notebooks. It can help avoid duplicates and detect if a Colab-ready notebook already exists.

Run the Python cell below to list any `.ipynb` files in the repo and preview the first few lines of any candidate notebook.

In [ ]:
# Cell 2: Search for notebooks in repo
import os
from pathlib import Path

repo_root = Path('/content/GhanaSegNet') if Path('/content/GhanaSegNet').exists() else Path('.')
notebooks = [str(p) for p in Path('.').rglob('*.ipynb')]
print(f'Found {len(notebooks)} notebook(s) in repo root:')
for nb in notebooks:
    print('-', nb)

# Preview the first candidate (if any)
if notebooks:
    with open(notebooks[0], 'r', encoding='utf-8') as f:
        first_lines = ''.join([next(f) for _ in range(5)])
    print('\nPreview of', notebooks[0], ':')
    print(first_lines)

## 2) Prepare Colab-compatible notebook metadata

If you'd like to programmatically patch notebook metadata to be Colab-friendly (kernelspec and runtime), run the helper below. It uses nbformat to open and rewrite metadata.

Note: This step is optional; Colab generally handles metadata itself when opening a notebook from Drive.

In [ ]:
# Cell: helper to update notebook metadata to be Colab-friendly
try:
    import nbformat
    from nbformat import read, write, v4
except Exception as e:
    print('nbformat not available, you can pip install nbformat if you want to run this cell')


def patch_notebook_metadata(nb_path: str):
    nb = nbformat.read(nb_path, as_version=4)
    nb.metadata.setdefault('kernelspec', {})
    nb.metadata['kernelspec'].update({
        'name': 'python3',
        'display_name': 'Python 3'
    })
    nb.metadata.setdefault('colab', {})
    nb.metadata['colab'].update({'name': Path(nb_path).name})
    nbformat.write(nb, nb_path)
    print('Patched metadata for', nb_path)

# Example usage (uncomment to run):
# patch_notebook_metadata('Colab_Evaluate_and_Package.ipynb')

In [ ]:
# Verification & optional automatic run:
# This cell will:
#  - list copied JSONs and Drive-hosted checkpoint files,
#  - if JSONs are present, automatically run Path A analysis,
#  - if no JSONs are present and AUTO_RUN_PATH_B is True, copy checkpoints per-model and run Path B evaluations,
#  - finally package results and copy the zip back to Drive.
from pathlib import Path
import os
import shutil

# Configuration: toggle auto-running Path B (full evaluation) when no JSONs are found.
# Keep False by default to avoid long compute without explicit consent.
AUTO_RUN_PATH_B = False

RUNTIME_RESULTS = Path('/content/GhanaSegNet/results')
# Resolve DRIVE_CHECKPOINTS_DIR robustly: try globals, then DRIVE_PROJECT_DIR, then a DRIVE_ROOT-based default
drv_ck = globals().get('DRIVE_CHECKPOINTS_DIR', None)
if not drv_ck:
    # try DRIVE_PROJECT_DIR if available
    drv_ck = globals().get('DRIVE_PROJECT_DIR', None)
if drv_ck:
    DRIVE_CKPTS = Path(drv_ck)
else:
    # fallback to common Drive locations
    DRIVE_ROOT = globals().get('DRIVE_ROOT', '/content/drive/My Drive')
    DRIVE_CKPTS = Path(DRIVE_ROOT) / 'GhanaSegNet'
DRIVE_ROOT = globals().get('DRIVE_ROOT', '/content/drive/My Drive')

# Gather JSONs copied to runtime
jsons = sorted([p.name for p in RUNTIME_RESULTS.glob('*.json')])

# Gather checkpoint file names from Drive (do not copy here)
ckpt_files = []
if DRIVE_CKPTS.exists():
    for model_dir in sorted(DRIVE_CKPTS.iterdir()):
        if model_dir.is_dir():
            for p in model_dir.glob('*'):
                # record relative path so output is readable
                try:
                    ckpt_files.append(str(p.relative_to(DRIVE_CKPTS)))
                except Exception:
                    ckpt_files.append(str(p))

print('Sample JSONs (first 50):')
for n in jsons[:50]:
    print(' -', n)
print(f'Total JSON files: {len(jsons)}')

print('\nSample checkpoint files (first 50) (from Drive):')
for n in ckpt_files[:50]:
    print(' -', n)
print(f'Total checkpoint files: {len(ckpt_files)}')

# If JSONs are present, run Path A analysis automatically (quick path)
if len(jsons) > 0:
    print('\nFound JSONs — running Path A analysis (quick).')
    try:
        # Ensure we're in the workspace with the repo code available
        os.chdir('/content/GhanaSegNet') if Path('/content/GhanaSegNet').exists() else None
        print('Running analysis/model_comparison_analysis.py')
        os.system('python analysis/model_comparison_analysis.py')
        print('Running analysis/compute_val_iou_stats.py')
        os.system('python analysis/compute_val_iou_stats.py')
        print('Path A analysis complete. Results are in results/ directory.')
    except Exception as e:
        print('Error while running Path A analysis:', e)

    # Package results to Drive
    OUT_ZIP = Path('/content/results_per_class_summary_jsons.zip')
    if OUT_ZIP.exists():
        OUT_ZIP.unlink()
    shutil.make_archive(str(OUT_ZIP).replace('.zip',''), 'zip', 'results')
    DRIVE_OUT = os.path.join(DRIVE_ROOT, 'GhanaSegNet_Results')
    Path(DRIVE_OUT).mkdir(parents=True, exist_ok=True)
    shutil.copy(str(OUT_ZIP), os.path.join(DRIVE_OUT, 'results_per_class_summary_jsons.zip'))
    print('Packaged results.zip to', os.path.join(DRIVE_OUT, 'results_per_class_summary_jsons.zip'))

else:
    print('\nNo JSONs found in runtime results.')
    if AUTO_RUN_PATH_B:
        print('AUTO_RUN_PATH_B is True — running full Path B across all model folders found in Drive (one-by-one).')
        # For each model folder in Drive, copy its checkpoints into runtime and run evaluation, then copy per-model outputs back to Drive.
        drive_project = Path(DRIVE_PROJECT_DIR)
        models = [p for p in drive_project.iterdir() if p.is_dir()]
        if not models:
            print('No model folders found under DRIVE_PROJECT_DIR:', DRIVE_PROJECT_DIR)
        for model_dir in models:
            model_name = model_dir.name
            print(f'\nProcessing model: {model_name}')
            dest_ckpt = Path('/content/GhanaSegNet/checkpoints') / model_name
            if dest_ckpt.exists():
                print('Runtime checkpoint dest already exists; removing and re-copying')
                shutil.rmtree(dest_ckpt)
            dest_ckpt.mkdir(parents=True, exist_ok=True)
            count = 0
            for ext in ('*.pth','*.pt','*.ckpt'):
                for ck in model_dir.glob(ext):
                    shutil.copy(ck, dest_ckpt / ck.name)
                    count += 1
            print(f'Copied {count} checkpoints for {model_name} to', dest_ckpt)
            # Run evaluation for this model (assumes evaluation script writes to results/)
            cmd = f'python analysis/evaluate_checkpoints_per_class.py --checkpoints checkpoints/{model_name} --out results/{model_name}_per_class.json'
            print('Running:', cmd)
            os.system(cmd)
            # Optionally remove the copied checkpoints to free space
            shutil.rmtree(dest_ckpt)
        # After loop, package results
        OUT_ZIP = Path('/content/results_per_class_summary_jsons.zip')
        if OUT_ZIP.exists():
            OUT_ZIP.unlink()
        shutil.make_archive(str(OUT_ZIP).replace('.zip',''), 'zip', 'results')
        DRIVE_OUT = os.path.join(DRIVE_ROOT, 'GhanaSegNet_Results')
        Path(DRIVE_OUT).mkdir(parents=True, exist_ok=True)
        shutil.copy(str(OUT_ZIP), os.path.join(DRIVE_OUT, 'results_per_class_summary_jsons.zip'))
        print('Packaged and copied results to Drive at', os.path.join(DRIVE_OUT, 'results_per_class_summary_jsons.zip'))
    else:
        print('AUTO_RUN_PATH_B is False. If you want to run full evaluations on checkpoints, either set AUTO_RUN_PATH_B = True and re-run this cell, or use the helper cell to copy a single model checkpoint and run Path B manually.')

In [ ]:
# Diagnostic: locate Drive project and count JSONs/checkpoints
from pathlib import Path
import os

print('--- Drive variable snapshot (from globals) ---')
for name in ('DRIVE_ROOT','DRIVE_PROJECT_DIR','DRIVE_RESULTS_DIR','DRIVE_CHECKPOINTS_DIR'):
    print(name, '=', globals().get(name, None))

# Helper to safely inspect a path
def inspect_path(p):
    p = Path(p)
    out = {'path': str(p), 'exists': p.exists(), 'is_dir': p.is_dir(), 'json_count': 0, 'ckpt_count': 0, 'sample': []}
    if p.exists() and p.is_dir():
        try:
            out['json_count'] = sum(1 for _ in p.rglob('*.json'))
            out['ckpt_count'] = sum(1 for _ in p.rglob('*.pth')) + sum(1 for _ in p.rglob('*.pt')) + sum(1 for _ in p.rglob('*.ckpt'))
            out['sample'] = [str(x.relative_to(p)) for i,x in enumerate(p.rglob('*')) if i < 20]
        except Exception as e:
            out['error'] = str(e)
    return out

# Inspect the main configured paths (if set)
candidates = []
for var in ('DRIVE_RESULTS_DIR','DRIVE_CHECKPOINTS_DIR','DRIVE_PROJECT_DIR','DRIVE_ROOT'):
    v = globals().get(var, None)
    if v:
        candidates.append((var, v))

for var, path in candidates:
    info = inspect_path(path)
    print('\n==', var, '=>', info['path'], '==')
    print(' exists:', info['exists'], ' is_dir:', info['is_dir'])
    if info.get('error'):
        print(' error inspecting path:', info['error'])
    else:
        print(' json_count:', info['json_count'], ' ckpt_count:', info['ckpt_count'])
        print(' sample entries (up to 20):')
        for s in info['sample'][:20]:
            print('  -', s)

# If nothing useful, run a targeted search under DRIVE_ROOT for likely folders
ROOT = Path(globals().get('DRIVE_ROOT', '/content/drive'))
print('\nSearching under DRIVE_ROOT for likely project/result folders (this may take a few seconds)')
found = []
if ROOT.exists():
    for p in ROOT.rglob('*'):
        if p.is_dir() and any(k in p.name.lower() for k in ('ghanas','ghana','ghanasegnet','ghanasgnet','ghanasgnet_results','ghana_seg','ghana_segnet','ghanasgnet_results','results')):
            found.append(p)
    found = sorted(set(found))
    print('Found', len(found), 'candidate directories:')
    for p in found[:50]:
        js = sum(1 for _ in p.rglob('*.json')) if p.exists() else 0
        ck = (sum(1 for _ in p.rglob('*.pth')) + sum(1 for _ in p.rglob('*.pt')) + sum(1 for _ in p.rglob('*.ckpt'))) if p.exists() else 0
        print(' -', str(p), f'(jsons={js}, ckpts={ck})')
else:
    print('DRIVE_ROOT does not exist:', ROOT)

print('\nDiagnostic complete. If you see a candidate path that holds your JSONs or checkpoints, set DRIVE_PROJECT_DIR to that folder (or update DRIVE_RESULTS_DIR/DRIVE_CHECKPOINTS_DIR) and re-run the normalization and verification cells.')

# End diagnostic cell

In [ ]:
# Helper: list model folders and checkpoint/json counts under the detected Drive project
from pathlib import Path
import os

# Resolve a candidate project path (try DRIVE_PROJECT_DIR, then DRIVE_CHECKPOINTS_DIR, then DRIVE_ROOT)
candidate = globals().get('DRIVE_PROJECT_DIR') or globals().get('DRIVE_CHECKPOINTS_DIR') or globals().get('DRIVE_ROOT') or '/content/drive/My Drive'
proj = Path(candidate)
print('Listing model folders under:', proj)

if not proj.exists():
    print('Path does not exist. Did you mount Drive and set DRIVE_PROJECT_DIR?')
else:
    subdirs = sorted([p for p in proj.iterdir() if p.is_dir()])
    if not subdirs:
        print('No subfolders found under', proj)
    else:
        print(f'Found {len(subdirs)} subfolders under {proj}:')
        print()
        for p in subdirs:
            try:
                json_count = sum(1 for _ in p.rglob('*.json'))
                ckpt_count = sum(1 for _ in p.rglob('*.pth')) + sum(1 for _ in p.rglob('*.pt')) + sum(1 for _ in p.rglob('*.ckpt'))
            except Exception as _e:
                json_count = 0
                ckpt_count = 0
            print(f'- {p.name:30} jsons={json_count:4}  ckpts={ckpt_count:4}  path={str(p)}')

        print('
If you see a model you want to evaluate with Path B, set DRIVE_PROJECT_DIR to the parent folder above (or set DRIVE_CHECKPOINTS_DIR), then use the helper that copies a single checkpoint into the runtime.')